# Demo: OpenAI Large Language Model Chain of Thoughts Demo

In this demo, we use the GPT-3 model to analyze natural queries, retrieve additional information using a knowledge base, and answer questions.  
2. Utilizing Azure AI Search: We examine how the Wikipedia search engine created on the first day can be utilized.

In [1]:
import os
import json
import requests
from openai import AzureOpenAI
from dotenv import load_dotenv
load_dotenv()

client = AzureOpenAI(
    azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT","").strip(),
    api_key        = os.getenv("AZURE_OPENAI_API_KEY"),
    api_version    = os.getenv("OPENAI_API_VERSION")
)

deployment_name = os.getenv('DEPLOYMENT_NAME')

## 2. How to search based on your own data using AI Search  

The RAG (Retrieval-Augmented Generation) model combines an information retrieval system with a language model to retrieve relevant data for a given question and generate an answer based on that information. In this process, the Azure AI Search service plays a pivotal role.

1. Search efficiency: Azure AI Search provides the capability to quickly retrieve relevant information from large datasets. The RAG model uses such search engines to efficiently find documents or data related to the given question, enabling it to generate more accurate answers.  

2. Accuracy and reliability of information: Azure AI Search enhances the precision of search results using various optimization techniques and algorithms. This helps the RAG model base its answers on more reliable information.  

3. Scalability and availability: As a cloud-based service, Azure AI Search is highly scalable and accessible from anywhere at any time. The RAG model leverages this cloud service to meet the search needs for datasets of varying sizes and scopes.  

4. Customizability and adaptability: Azure AI Search allows users to finely tune and optimize the search algorithm. This enables the RAG model to tailor its search capabilities to specific domains or question types, which contributes to enhancing the overall quality of the answers.  

> The following processes assume prior configurations have been completed:
- The search engine indexes have been created using the Wikipedia document file, which includes vector data provided by OpenAI, uploaded to [http://azure.studydev.com/openai/vector_sample.csv](http://azure.studydev.com/openai/vector_sample.csv).
- Tutorial video on how to index vector data into the Azure AI Search engine: [https://www.youtube.com/watch?v=XsqPjHYZD04](https://www.youtube.com/watch?v=XsqPjHYZD04)

In [2]:
# Azure Cognitive Search
# Create an SDK client
# endpoint = os.getenv("OPENAI_API_ENDPOINT")
from azure.core.credentials import AzureKeyCredential
from azure.search.documents import SearchClient
from azure.search.documents.models import VectorizedQuery

endpoint    = os.getenv('AZURE_AI_SEARCH_ENDPOINT')
credential  = AzureKeyCredential(os.getenv('AZURE_AI_SEARCH_API_KEY'))
index_name  = os.getenv('AZURE_AI_SEARCH_INDEX')
api_version = os.getenv('AZURE_AI_SEARCH_API_VERSION')
embedding_model_name = os.getenv("DEPLOYMENT_EMBEDDING_NAME")

search_client = SearchClient(endpoint=endpoint, index_name=index_name, credential=credential, api_version=api_version)

question     = "What are the differences between April and August?"
embedding    = client.embeddings.create(input = question, model=embedding_model_name).data[0].embedding
vector_query = VectorizedQuery(vector=embedding, k_nearest_neighbors=3, fields="content_vector", exhaustive=True)

results = search_client.search(  
    search_text=None,  
    vector_queries= [vector_query],
    select=["id", "title", "text"],
)  


### Collected information for further processing

[Samples for Azure AI Search client library for Python](https://github.com/Azure/azure-sdk-for-python/tree/main/sdk/search/azure-search-documents/samples)

In [3]:
grounding = ""
for result in results:  
    # print(f"Title: {result['title']}")  
    # print(f"Score: {result['@search.score']}")  
    # print(f"Text:  {result['text']}")  
    # print(f"id:    {result['id']}\n")  
    grounding = grounding + " \n " + result['text']

In [4]:
input = question
prompt=f'''Answer the following questions as best you can. You have access to the following tools:
Question: {input}
Gronding data: {grounding}
Thought:'''


In [5]:
print(prompt)

Answer the following questions as best you can. You have access to the following tools:
Question: What are the differences between April and August?
Gronding data:  
 August (Aug.) is the eighth month of the year in the Gregorian calendar, coming between July and September. It has 31 days. It is named after the Roman emperor Augustus Caesar. August does not begin on the same day of the week as any other month in common years, but begins on the same day of the week as February in leap years. August always ends on the same day of the week as November. The Month This month was first called Sextilis in Latin, because it was the sixth month in the old Roman calendar. The Roman calendar began in March about 735 BC with Romulus. October was the eighth month. August was the eighth month when January or February were added to the start of the year by King Numa Pompilius about 700 BC. Or, when those two months were moved from the end to the beginning of the year by the decemvirs about 450 BC (Ro

### Step 3: GPT-3: What do I need to do to answer this question?

In [6]:
response = client.chat.completions.create(
    model=deployment_name,
    messages = [
        {"role":"system", "content":"You are a helpful assistant. If you have data that can be expressed in a table, use Markdown to print it as a table. You can only use English language."},
        {"role":"user","content": prompt}
    ],
    max_tokens=600
)

print(response.choices[0].message.content)

To compare April and August, I will create a table highlighting their differences:

| Feature                | April                              | August                             |
|------------------------|------------------------------------|------------------------------------|
| Order in the Year      | 4th month                         | 8th month                         |
| Number of Days         | 30 days                           | 31 days                           |
| Named After            | Unclear (possibly from "aperire" or Aphrodite) | Augustus Caesar                   |
| Zodiac Signs           | Aries (March 21-April 20), Taurus (April 21-May 20) | Leo (July 22-August 21), Virgo (August 22-September 21) |
| Birthstones            | Diamond                           | Peridot                            |
| Birth Flowers          | Sweet Pea, Daisy                  | Gladiolus                          |
| Seasonal Position      | Spring in Northern Hemisphere, Autumn 